# Code to run the different models

## Initial import of packages

In [7]:
from data_handler import get_dataset
from data_utils import *
import matplotlib.pyplot as plt

from models.model_utils import get_model, get_model_args
from models.armodels import get_baselines
from preprocess_data import *
from models.gcnlstm import get_gcnlstm
from data_utils import *
from graph_utils import *

import tensorflow as tf
from keras.optimizers import Adam

## Data setup

In [8]:
dataset = "Palo Alto"
model_name = "convlstm"

prediction_horizon = 7
num_lags = 60

In [9]:
data = get_dataset(dataset)

/Users/frederikhyttel/miniforge3/envs/ml/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (29,30,32) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
[-122.163289  -122.1531172 -122.1429454 -122.1327736 -122.1226018
 -122.11243  ]
[37.422054  37.4275282 37.4330024 37.4384766 37.4439508 37.449425 ]


KeyboardInterrupt: 

## Get baseline predictions

In [5]:
Var_forecast, AR_forecast = get_baselines(prediction_horizon, get_raster_map(data, normalized=True), data)

## Setup of CONV or CONV+LSTM models

In [4]:
X_train, y_train, X_test, y_test = process_data(data, model_name, prediction_horizon, num_lags)

In [6]:
model_args = get_model_args(model_name, prediction_horizon, X_train) 
model = get_model(model_name, model_args)

NameError: name 'X_train' is not defined

In [ ]:
model.summary()

In [5]:
model.compile(loss='mae',  optimizer=Adam(lr=model_args['lr']) ,metrics=["mse",'mae'])

hist = model.fit(X_train, y_train,epochs=1,
          batch_size=60, verbose=2, shuffle=True,
          validation_data=(X_test, y_test))

NameError: name 'model' is not defined

## Setup of T-GCN model

In [5]:
G, adj, X_train, y_train, X_test, y_test = setup_GCN(data, prediction_horizon, num_lags)

In [9]:
model = get_gcnlstm(prediction_horizon, num_lags, adj)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 47, 60)]          0         
_________________________________________________________________
tf.expand_dims (TFOpLambda)  (None, 47, 60, 1)         0         
_________________________________________________________________
reshape (Reshape)            (None, 47, 60)            0         
_________________________________________________________________
fixed_adjacency_graph_convol (None, 47, 64)            6096      
_________________________________________________________________
fixed_adjacency_graph_convol (None, 47, 64)            6352      
_________________________________________________________________
reshape_1 (Reshape)          (None, 47, 64, 1)         0         
_________________________________________________________________
permute (Permute)            (None, 64, 47, 1)         0     

In [10]:
model.compile(loss='mae',  optimizer=Adam(lr=1e-5) ,metrics=["mse",'mae'])

In [11]:
hist = model.fit(
    X_train,
    y_train,
    epochs=1,
    batch_size=16,
    shuffle=True,
    verbose=2,
    validation_data=(X_test, y_test),
)



Epoch 1/1000
211/211 - 3s - loss: 0.1934 - mse: 0.0680 - mae: 0.1934 - val_loss: 0.1198 - val_mse: 0.0367 - val_mae: 0.1198
Epoch 2/1000
211/211 - 2s - loss: 0.1738 - mse: 0.0584 - mae: 0.1738 - val_loss: 0.1198 - val_mse: 0.0365 - val_mae: 0.1198
Epoch 3/1000


KeyboardInterrupt: 